In [1]:
import numpy as np, sys
import numpy as np
import pickle
np.random.seed(1)

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

images, labels = (x_train[0:3000].reshape(3000,28*28) / 255,
                  y_train[0:3000])


one_hot_labels = np.zeros((len(labels),10))
for i,l in enumerate(labels):
    one_hot_labels[i][l] = 1
labels = one_hot_labels

x_test = x_test[:1000]
y_test = y_test[:1000]
test_images = x_test.reshape(len(x_test),28*28) / 255
test_labels = np.zeros((len(y_test),10))
for i,l in enumerate(y_test):
    test_labels[i][l] = 1

def tanh(x):
    return np.tanh(x)

def tanh2deriv(output):
    return 1 - (output ** 2)

def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

alpha, iterations = (2, 50)
pixels_per_image, num_labels = (784, 10)
batch_size = 128

input_rows = 28
input_cols = 28

kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) * 
               (input_cols - kernel_cols)) * num_kernels

# weights_0_1 = 0.02*np.random.random((pixels_per_image,hidden_size))-0.01
kernels = 0.02*np.random.random((kernel_rows*kernel_cols,
                                 num_kernels))-0.01

weights_1_2 = 0.2*np.random.random((hidden_size,
                                    num_labels)) - 0.1



def get_image_section(layer,row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)

for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1) # (128, 625, 3, 3)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
        kernel_output = flattened_input.dot(kernels)
        
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))

        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) == 
                               np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (labels[batch_start:batch_end]-layer_2)\
                        / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \
                        tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
    
    test_correct_cnt = 0

    for i in range(len(test_images)):

        layer_0 = test_images[i:i+1]
#         layer_1 = tanh(np.dot(layer_0,weights_0_1))
        layer_0 = layer_0.reshape(layer_0.shape[0],28,28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
                sects.append(sect)

        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)

        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)

        test_correct_cnt += int(np.argmax(layer_2) == 
                                np.argmax(test_labels[i:i+1]))
    if(j % 1 == 0):
        sys.stdout.write("\n"+ \
         "I:" + str(j) + \
         " \tTest-Acc:"+str(test_correct_cnt/float(len(test_images)))+\
         " \tTrain-Acc:" + str(correct_cnt/float(len(images))))
        
# save the model to disk
pickle.dump(weights_1_2, open('weights_1_2_model.sav', 'wb'))

pickle.dump(kernels, open('kernels_model.sav', 'wb'))


I:0 Test-Acc:0.027 Train-Acc:0.046
I:1 Test-Acc:0.062 Train-Acc:0.053
I:2 Test-Acc:0.299 Train-Acc:0.131
I:3 Test-Acc:0.495 Train-Acc:0.23966666666666667
I:4 Test-Acc:0.399 Train-Acc:0.33366666666666667
I:5 Test-Acc:0.015 Train-Acc:0.112
I:6 Test-Acc:0.027 Train-Acc:0.028333333333333332
I:7 Test-Acc:0.319 Train-Acc:0.11166666666666666
I:8 Test-Acc:0.587 Train-Acc:0.297
I:9 Test-Acc:0.671 Train-Acc:0.4523333333333333
I:10 Test-Acc:0.7 Train-Acc:0.5416666666666666
I:11 Test-Acc:0.73 Train-Acc:0.5753333333333334
I:12 Test-Acc:0.728 Train-Acc:0.5833333333333334
I:13 Test-Acc:0.676 Train-Acc:0.535
I:14 Test-Acc:0.6 Train-Acc:0.4056666666666667
I:15 Test-Acc:0.677 Train-Acc:0.41533333333333333
I:16 Test-Acc:0.738 Train-Acc:0.5103333333333333
I:17 Test-Acc:0.74 Train-Acc:0.5736666666666667
I:18 Test-Acc:0.752 Train-Acc:0.5793333333333334
I:19 Test-Acc:0.779 Train-Acc:0.604
I:20 Test-Acc:0.785 Train-Acc:0.632
I:21 Test-Acc:0.792 Train-Acc:0.6686666666666666
I:22 Test-Acc:0.806 Train-Acc:0.683

In [3]:
# load the model from disk
weights_1_2 = pickle.load(open('weights_1_2_model.sav', 'rb'))
kernels = pickle.load(open('kernels_model.sav', 'rb'))


def get_one_image_section(layer,row_from, row_to, col_from, col_to):
    section = layer[row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)
def predict(input_image):
    layer_0 = input_image
    layer_0 = layer_0.reshape(28,28)
    layer_0.shape

    sects = list()
    for row_start in range(layer_0.shape[0]-kernel_rows):
        for col_start in range(layer_0.shape[1] - kernel_cols):
            sect = get_one_image_section(layer_0,
                                     row_start,
                                     row_start+kernel_rows,
                                     col_start,
                                     col_start+kernel_cols)
            sects.append(sect)

    expanded_input = np.concatenate(sects,axis=1)
    es = expanded_input.shape
    flattened_input = expanded_input.reshape(es[0]*es[1],-1)

    kernel_output = flattened_input.dot(kernels)
    layer_1 = tanh(kernel_output.reshape(es[0],-1))
    layer_2 = np.dot(layer_1,weights_1_2)
    return layer_2

print('True labels: ', np.argmax(test_labels[2]))
print('prediction label: ', np.argmax(predict(test_images[2])))

True labels:  1
prediction label:  1
